In [1]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
#from tensorflow.contrib.rnn import GRUCell
from tensorflow.python.layers import core as layers_core

# __all__ = ["AttentionModel"]
print("TensorFlow Version", tf.__version__)

C:\Users\ericmlyang\AppData\Local\Continuum\anaconda3\envs\tf_cpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow Version 1.8.0


In [ ]:
"""tensorflow.python.layers:
layers 模塊提供用於深度學習的更高層次封裝的 API，

tf.layers 模塊提供的方法有：
    Input(…): 用於實例化一個輸入 Tensor，作為神經網絡的輸入。
    average_pooling1d(…): 一維平均池化層
    average_pooling2d(…): 二維平均池化層
    average_pooling3d(…): 三維平均池化層
    batch_normalization(…): 批量標準化層
    conv1d(…): 一維卷積層
    conv2d(…): 二維卷積層
    conv2d_transpose(…): 二維反捲積層
    conv3d(…): 三維卷積層
    conv3d_transpose(…): 三維反捲積層
    dense(…): 全連接層
    dropout(…): Dropout層
    flatten(…): Flatten層，即把一個 Tensor 展平
    max_pooling1d(…): 一維最大池化層
    max_pooling2d(…): 二維最大池化層
    max_pooling3d(…): 三維最大池化層
    separable_conv2d(…): 二維深度可分離卷積層
"""

# Batch Data Generation

In [2]:
x_input = pd.read_csv('plc_x_reduce.csv')
y_label = pd.read_csv('plc_y_reduce.csv')

In [3]:
def batch_generator(x_input,
                    y_label,
                    batch_size=1, 
                    seq_len=1):
    
    batchs = int(y_label.shape[0] / seq_len / batch_size)
    print(batchs)
    
    input_data = []
    target_data = []
    
    
    for i in range(batchs):
        x = np.zeros(shape=(batch_size, seq_len, x_input.shape[1]))
        y = np.zeros(shape=(batch_size, seq_len, y_label.shape[1]))
        for b in range(batch_size):

            x[b, :, :] = x_input[b*i : b*i+seq_len, :]
            y[b, :, :] = y_label[b*i : b*i+seq_len, :]

        input_data.append(x)
        target_data.append(y)              
    return input_data, target_data

# Model Class

In [103]:
class Seq2Seq:
    def __init__(self,
                 seq_max_len=1.,  
                 input_len=71.,
                 output_len=41.,
                 batch_size=1,
                 lstm_size=[128., 128., 128.],
                 learning_rate=0.005,
                 grad_clip=2.,
                 keep_prob=1.,
                 forward_only= None):
        
        
        self.seq_max_len = seq_max_len
        self.batch_size = batch_size
        self.batch_len = np.array([])
        self.input_len = input_len
        self.output_len = output_len
        self.lstm_size = lstm_size
        self.num_units = self.lstm_size[-1]
        self.num_layers = len(self.lstm_size)
        self.learning_rate = learning_rate
        self.grad_clip = grad_clip
        self.train_keep_prob = keep_prob
        
        self.batch_seq_len = np.int32(np.ones(shape=([self.batch_size])) * self.seq_max_len)
        
        tf.reset_default_graph() #Clears the default graph stack and resets the global default graph
        self.build_inputs()
        #self.build_embedding()
        self.build_encoder()
        self.build_decoder()
        self.build_loss()
        self.build_optimizer()
        self.saver = tf.train.Saver() #Saves and restores variables.
 
        

    def build_inputs(self):
        self.encoder_inputs = tf.placeholder(tf.float32, 
                                         shape=(self.batch_size, self.seq_max_len, self.input_len),
                                         name='inputs')
        self.targets = tf.placeholder(tf.float32,
                                         shape=(self.batch_size, self.seq_max_len, self.output_len),
                                         name='targets')
        self.decoder_inputs = tf.placeholder(tf.float32,
                                                shape=(self.batch_size, self.seq_max_len, self.output_len),  
                                                name='decoder_inputs')        
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
         
        '''if seq is different， we needd the sequence input :
        self.input_sequence_length = tf.placeholder(shape=([self.batch_size]), dtype=tf.int32, name='input_length')
        self.decoder_sequence_length = tf.placeholder(shape=([self.batch_size]), dtype=tf.int32, name='decoder_inputs_length')
        self.target_sequence_length = tf.placeholder(shape=([self.batch_size]), dtype=tf.float32, name='target_sequence_length')
        '''

    # Encoder Model==========================================================================
    def build_encoder(self):
        ''' Encoder Model'''
        def get_a_cell(lstm_size, keep_prop):
            lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
            drop = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=self.train_keep_prob)
            return drop

        with tf.variable_scope('encoder', initializer=tf.orthogonal_initializer()):
            encoder_cell = tf.nn.rnn_cell.MultiRNNCell(
                                 [get_a_cell(size, self.keep_prob) for size in self.lstm_size]
                                                      )
            self.initial_state = encoder_cell.zero_state(self.batch_size, tf.float32)
            # 透過dynamic_rnn對cell展開時間維度
            self.encoder_outputs, self.encoder_state  = tf.nn.dynamic_rnn(
                                                           encoder_cell, 
                                                           self.encoder_inputs,                                                    
                                                           initial_state=self.initial_state
                                                                          )

            
    # Decoder Model==============================================================================
    def build_decoder(self):
        decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(self.lstm_size[-1])
        #decoder_cell = tf.contrib.rnn.GRUCell(self.lstm_size[-1])
        
        '''Training Helper
        Time_major =False(default): [batch_size, max_seq_len, vector_len]
        Time_major =True : [max_seq_len, batch_size, vector_len]
        '''



        '''Project layer (output layer, full connecting layers)'''
        project_layer = layers_core.Dense(self.output_len, 
                                          kernel_initializer=tf.truncated_normal_initializer(mean=0.1,stddev=0.1), 
                                          name="output_projection")
        
        
        ''' Two decoder model:
        . training_decoder : for training & target as input
        . predict_decoder : for predecting & beam search as input
        '''
        '''1. Training decoder & output '''
        training_helper = tf.contrib.seq2seq.TrainingHelper(
                                     self.decoder_inputs, self.batch_seq_len, time_major=False)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(
                                              cell=decoder_cell,
                                              helper=training_helper,
                                              initial_state=self.encoder_state[-1],#self.encoder_state,
                                              output_layer=project_layer)
        train_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder, impute_finished=True)
        self.logits = train_outputs.rnn_output
        self.train_prediction = tf.sigmoid(self.logits, name='predictions')
        
        
        
        
        '''2. Predicting decoder & output (same parameter) '''
        #predicting_helper = tf.contrib.seq2seq.TrainingHelper(
        #                               self.encoder_state[-1], self.batch_seq_len, time_major=False)
        
        embedding = tf.get_variable('embedding', [self.output_len, 1])
        start_tokens = np.int32(np.ones(shape=([self.batch_size])))
        end_token = 0
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding=embedding, start_tokens=start_tokens, end_token=end_token)

        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(cell=decoder_cell,
                                                             helper=predicting_helper,
                                                             initial_state=self.encoder_state[-1],
                                                             output_layer=project_layer)
        predicting_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder, impute_finished=True)
        predict_logits = train_outputs.rnn_output
        self.final_prediction = tf.sigmoid(predict_logits, name='predictions')
        
        



 

    # Loss & Optimizer ==============================================================================
    def build_loss(self):
        with tf.name_scope('loss'):
            #self.y_reshaped = tf.reshape(self.targets,  self.logits.get_shape())
            #self.loss =tf.losses.mean_squared_error(predictions=self.logits, labels=self.targets)
            self.loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.logits, labels=self.targets)
            self.loss = tf.reduce_mean(self.loss)

    def build_optimizer(self):
        # Using "clipping" gradients
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), self.grad_clip)
        train_op = tf.train.AdamOptimizer(self.learning_rate)
        self.optimizer = train_op.apply_gradients(zip(grads, tvars))
 

        
        
        

    # Training===============================================================================    
    def train(self, x, y, iters=10, save_path='./models', save_every_n=200, log_every_n=200):
        self.session = tf.Session()
        with self.session as sess:
            sess.run(tf.global_variables_initializer())
            # Train network

            sess.run(self.initial_state)
            #sess.run(self.d_initial_state)
            
            
            for ite in range(iters):
                step = 0
                print('iters',ite)
                for i in range(len(x)):
                    step += 1
                    start = time.time()
                    
                    feed = {self.encoder_inputs: x[i], 
                            self.targets: y[i],
                            self.decoder_inputs : y[i],
                            self.keep_prob: self.train_keep_prob}
                            #self.initial_state: new_state}
                    
                    batch_loss, new_state, pred, target = sess.run([self.loss,
                                                      self.optimizer,
                                                      self.train_prediction,
                                                      self.targets],
                                                      feed_dict=feed)
                    # print result
                    #self.print_result(x[i], y[i])
                    end = time.time()
                    
                    # control the print lines
                    if step % log_every_n == 0:
                        print("=======================================================\n")
                        print('step: {} in iter: {}/{}... '.format(step, ite+1, iters),
                              'loss: {:.10f}... '.format(batch_loss),
                              '{:.4f} sec/batch'.format((end - start)))

                    if (step % save_every_n == 0):
                        self.saver.save(sess, os.path.join(save_path, 'model'), global_step=step)
                        self.jodge(pred, target)
                        #print("Target: \n", target)
                        #print("PRED: \n",pred)
                # print(tf.shape(self.encoder_state), tf.shape(self.d_initial_state))
                # print(np.array(state).shape)
                # print(np.array(self.d_initial_state).shape, self.d_initial_state)
                
    
    
    def predict(self, x, y):
        self.session = tf.Session()
        with self.session as sess:
            feed = {self.encoder_inputs: x, 
                    self.keep_prob: 1.}
            predict = sess.run([self.final_prediction], feed_dict=feed)
            return predict
            
                        
    
    
                
    # 
    def jodge(self, pred, target):
        pred = np.array(pred)
        pred = np.array(pred >= 0.5).astype(int)

        result = np.abs(pred - target)

        for i in range(result.shape[0]):
            np.savetxt("result_" + str(i) + ".csv", result[i], delimiter=',')
        return pred

# Train Model

In [104]:
batch_size = 5
seq_len = 5
lstm_size=[128., 128., 128.]
learning_rate=0.005

iters=1

In [105]:
model_path = './seq2seq_models'

if os.path.exists(model_path) is False:
    os.makedirs(model_path)
    
input_data, target_data = batch_generator(x_input.values, 
                                          y_label.values, 
                                          batch_size=batch_size, 
                                          seq_len=seq_len)

model = Seq2Seq(batch_size=batch_size, 
                seq_max_len=seq_len,
                lstm_size=lstm_size,
                learning_rate=learning_rate)

model.train(input_data, 
            target_data,
            iters=iters,
            save_path=model_path,
            save_every_n=1000,
            log_every_n =200
            )

2119
iters 0

step: 200 in iter: 1/1...  loss: 0.1096691713...  0.0140 sec/batch

step: 400 in iter: 1/1...  loss: 0.0135252718...  0.0080 sec/batch

step: 600 in iter: 1/1...  loss: 0.0053684143...  0.0080 sec/batch

step: 800 in iter: 1/1...  loss: 0.0132287201...  0.0090 sec/batch

step: 1000 in iter: 1/1...  loss: 0.0106951119...  0.0080 sec/batch

step: 1200 in iter: 1/1...  loss: 0.0001148187...  0.0080 sec/batch

step: 1400 in iter: 1/1...  loss: 0.0189563818...  0.0070 sec/batch

step: 1600 in iter: 1/1...  loss: 0.0004311473...  0.0070 sec/batch

step: 1800 in iter: 1/1...  loss: 0.0004814954...  0.0100 sec/batch

step: 2000 in iter: 1/1...  loss: 0.0008978583...  0.0080 sec/batch


In [106]:
model.predict(input_data[10], target_data[10])

InvalidArgumentError: You must feed a value for placeholder tensor 'decoder_inputs' with dtype float and shape [5,5,41]
	 [[Node: decoder_inputs = Placeholder[dtype=DT_FLOAT, shape=[5,5,41], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'decoder_inputs', defined at:
  File "C:\Users\ericmlyang\AppData\Local\Continuum\anaconda3\envs\tf_cpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\ericmlyang\AppData\Local\Continuum\anaconda3\envs\tf_cpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\ericmlyang\AppData\Local\Continuum\anaconda3\envs\tf_cpu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ericmlyang\AppData\Local\Continuum\anaconda3\envs\tf_cpu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\ericmlyang\AppData\Local\Continuum\anaconda3\envs\tf_cpu\lib\si